In [1]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

# import pdfkit
import html
from copy import deepcopy
import re

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

### Setup Model

In [3]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

# conv_model = ChatOpenAI(model="ft:gpt-3.5-turbo-0613:yield-inc::8LLEaLUZ")
conv_model = ChatOpenAI(model="gpt-4-0613")
conv_model.temperature = 0.8

#### DBBench Tasks

In [25]:
import re
from tqdm.auto import tqdm 
from datasets import load_dataset
raw_dataset = load_dataset("THUDM/AgentInstruct", split="db")

task_environment_list = []
for turn in raw_dataset:
    task_environment_list.append(turn['conversations'][2]['value'])

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [26]:
system_prompt = \
"""You are given 5 examples of a task and an SQL database description. You have to make 5 more new tasks with a comprehensive description of the SQL database. 
Note you have to be diverse in innovating the SQL database environment and the task to perform. Pls make sure that the SQL database you create should be realistic. """

more_task_environment_list =[]
for _ in tqdm(range(100)):
    task_environment = random.sample(task_environment_list, 5)
    user_message_str = ""
    for i, (t_e) in enumerate(task_environment):
        t, e = t_e.split('\n')[0], t_e.split('\n')[1] 
        user_message_str += f"Task{i+1}: {t}\nEnvironment{i+1}: {e}\n\n"
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message_str),]
    response = model.predict_messages(messages)
    
    print(response.content)
    print('-'*50)
    
    pattern = r"(Task\d+:.*?)(?=\n\n|\Z)"
    tasks_and_envs = re.findall(pattern, response.content, re.DOTALL)
    task_env_pairs = [tuple(re.sub(r"Task\d+: |Environment\d+: ", "", te).split('\n', 1)) for te in tasks_and_envs]
    more_task_environment_list.extend(task_env_pairs)

  0%|          | 0/100 [00:00<?, ?it/s]

Task6: Query to find the total sales made by the employee 'John Doe'.
Environment6: There are 2 tables involved in this task. The first table is named 'employees' with headers 'emp_id', 'first_name', 'last_name', 'email', 'phone_number'. The second table is named 'sales' with headers 'sale_id', 'emp_id', 'product_id', 'quantity', 'sale_date', 'total_price'. 

Task7: Show the list of books that are currently issued and not returned.
Environment7: The library database has two tables. The first table is 'books' with headers 'book_id', 'title', 'author', 'genre', 'publish_date'. The second table is 'issue_records' with headers 'record_id', 'book_id', 'member_id', 'issue_date', 'return_date'. 

Task8: Find the average age of patients diagnosed with 'Influenza'.
Environment8: The hospital database has two tables. The first table is named 'patients' with headers 'patient_id', 'first_name', 'last_name', 'birth_date', 'gender', 'contact'. The second table is named 'diagnoses' with headers 'diag

In [31]:
more_task_environment_list__ = []
for m in more_task_environment_list:
    if len(m)!=5:print('no')
    more_task_environment_list__.extend(m)

no
no
no
no
no


In [30]:
len(more_task_environment_list__)

475

In [32]:
import pickle

with open('dbbench-more_task_environment_list.pkl', 'wb') as file:
    pickle.dump(more_task_environment_list__, file)

In [33]:
with open('dbbench-more_task_environment_list.pkl', 'rb') as file:
    more_task_environment_list__ = pickle.load(file)

### DBBench Environment

In [35]:
environment_prompt_template = '''
Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below

Initial Database state:
{}

First command:
{}

Please only respond in rawMySQL format (with no extra formatting or commentary) for a user of  mysql-connector-python, for example, if the result is 59.555, the result would be presented as [('59.555',)]. After responding, end your response.
'''

mysql_agent_prompt = '''I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question. You have to explain the problem and your solution to me and write down your thoughts. After thinking and explaining thoroughly, every round you can choose to operate or to answer. your operation should be like this: Action: Operation ```sql SELECT * FROM table WHERE condition; ``` You MUST put SQL in markdown format without any other comments. Your SQL should be in one line. Every time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you and give you the output. If you are done operating, and you want to commit your final answer, then write down: Action: Answer Final Answer: ["ANSWER1", "ANSWER2", ...] DO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer. Your answer should be accurate. Your answer must be exactly the same as the correct answer. If the question is about modifying the database, then after done operation, your answer field can be anything. If your response cannot match any pattern I mentioned earlier, you will be judged as FAIL immediately. Your input will be raw MySQL response, you have to deal with it by yourself.'''

In [36]:
task = '''How many French drivers who obtain the laptime less than 01:00.00? There are 2 tables involved with this task. The name of the 1st table is drivers, and the headers of this table are driverId,driverRef,number,code,forename,surname,dob,nationality,url. The name of the 2nd table is lapTimes, and the headers of this table are raceId,driverId,lap,position,time,milliseconds.'''
agent_messages = [
    HumanMessage(content=mysql_agent_prompt),
    AIMessage(content="Ok."),
    HumanMessage(content=task)
]


In [37]:
agent_response = conv_model.predict_messages(agent_messages)

In [38]:
agent_response.content

"This problem requires us to find the number of French drivers who have a laptime less than 01:00.00. To answer this question, we need to join the 'drivers' and 'lapTimes' tables on the 'driverId' field. After joining, we need to apply filter conditions where 'nationality' is 'French' and 'time' is less than '01:00.00'. To get the count, we can use the COUNT() function.\n\nHere is the SQL statement that represents the above explanation:\n\n```sql\nSELECT COUNT(DISTINCT d.driverId) \nFROM drivers d \nINNER JOIN lapTimes l \nON d.driverId = l.driverId \nWHERE d.nationality = 'French' AND l.time < '01:00.00';\n```"

In [39]:
import re
first_sql_block = re.search(r"```sql(.*?)```", agent_response.content, re.DOTALL)
if first_sql_block:
    sql_code = first_sql_block.group(1).strip()
else:
    sql_code = "No SQL code block found"
sql_code

"SELECT COUNT(DISTINCT d.driverId) \nFROM drivers d \nINNER JOIN lapTimes l \nON d.driverId = l.driverId \nWHERE d.nationality = 'French' AND l.time < '01:00.00';"

In [40]:
environment_information = task.split("?")[1].strip()

In [41]:
environment_prompt = environment_prompt_template.format(environment_information, sql_code)

environment_messages = [
    HumanMessage(content=environment_prompt)
]

In [42]:
environment_result = model.predict_messages(environment_messages)

In [43]:
environment_result.content

"[('23',)]"

In [44]:
agent_messages.append(HumanMessage(content=environment_result.content))

In [45]:
num_turns = 10

for i in range(num_turns):
    agent_response = conv_model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_result = agent_response.content.split("Action:")[1].strip()
    environment_messages.append(HumanMessage(content=agent_result))
    agent_messages.append(agent_response)
    if "Final Answer:" in agent_result:
        break
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    print(environment_result.content)
    agent_messages.append(HumanMessage(content=environment_result.content))

The question demands to find the number of French drivers who have a lap time of less than 1 minute. For this, we need to combine data from both the drivers and lapTimes tables. We have to join these two tables based on the driverId, then filter by nationality (French), and finally check if the time is less than 1 minute. 

Let's start by joining the two tables. We will use an INNER JOIN operation on the driverId of both tables. This will allow us to see lap time information for each French driver in the database. 

Action: Operation
```sql
SELECT drivers.driverId, drivers.nationality, lapTimes.time 
FROM drivers 
INNER JOIN lapTimes ON drivers.driverId = lapTimes.driverId 
WHERE drivers.nationality = 'French'
```
Operation
```sql
INSERT INTO drivers(driverId,driverRef,number,code,forename,surname,dob,nationality,url) 
VALUES (9999, 'vettel', 5, 'VET', 'Sebastian', 'Vettel', '1987-07-03', 'German','http://en.wikipedia.org/wiki/Sebastian_Vettel')
```
Sorry, I can't accept your operation

IndexError: list index out of range

#### Get tasks from AgentInstruct Data

In [41]:
import pandas as pd
df = pd.read_parquet('~/spikes/notebook_spikes/agentinstruct/db-00000-of-00001-916a87c4725da8c0.parquet')
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\harsh/spikes/notebook_spikes/agentinstruct/db-00000-of-00001-916a87c4725da8c0.parquet'

In [44]:
from datasets import load_dataset
raw_dataset = load_dataset("THUDM/AgentInstruct", split="db")
dataset = []
for ds in raw_dataset:
    dataset.append(ds)
df = pd.DataFrame(dataset)

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [45]:
print(df.iloc[2]["conversations"][2]['value'])

Among the cards in the set "Hauptset Zehnte Edition", how many of them are designed by Adam Rex?
There are 2 tables involved with this task. The name of the 1st table is cards, and the headers of this table are id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,edhrecRank,faceConvertedManaCost,faceName,flavorName,flavorText,frameEffects,frameVersion,hand,hasAlternativeDeckLimit,hasContentWarning,hasFoil,hasNonFoil,isAlternative,isFullArt,isOnlineOnly,isOversized,isPromo,isReprint,isReserved,isStarter,isStorySpotlight,isTextless,isTimeshifted,keywords,layout,leadershipSkills,life,loyalty,manaCost,mcmId,mcmMetaId,mtgArenaId,mtgjsonV4Id,mtgoFoilId,mtgoId,multiverseId,name,number,originalReleaseDate,originalText,originalType,otherFaceIds,power,printings,promoTypes,purchaseUrls,rarity,scryfallId,scryfallIllustrationId,scryfallOracleId,setCode,side,subtypes,supertypes,tcgplayerProductId,text,toughness,ty

In [46]:
def process(data):
    return (data['conversations'][2]['value'], '\n'.join(data['conversations'][2]['value'].split("\n")[1:]))

In [47]:
process(df.iloc[104])

('How many turnovers per game did the assist champion had in the 2003 NBA season?\nThere are 2 tables involved with this task. The name of the 1st table is players, and the headers of this table are playerID,useFirst,firstName,middleName,lastName,nameGiven,fullGivenName,nameSuffix,nameNick,pos,firstseason,lastseason,height,weight,college,collegeOther,birthDate,birthCity,birthState,birthCountry,highSchool,hsCity,hsState,hsCountry,deathDate,race. The name of the 2nd table is players_teams, and the headers of this table are id,playerID,year,stint,tmID,lgID,GP,GS,minutes,points,oRebounds,dRebounds,rebounds,assists,steals,blocks,turnovers,PF,fgAttempted,fgMade,ftAttempted,ftMade,threeAttempted,threeMade,PostGP,PostGS,PostMinutes,PostPoints,PostoRebounds,PostdRebounds,PostRebounds,PostAssists,PostSteals,PostBlocks,PostTurnovers,PostPF,PostfgAttempted,PostfgMade,PostftAttempted,PostftMade,PostthreeAttempted,PostthreeMade,note.',
 'There are 2 tables involved with this task. The name of the 1s

In [48]:
task_and_environment = df.apply(process, axis=1).tolist()

In [49]:
task_and_environment[200]

('Please list down ID of movies acted by top 5 actors based on actor rating.\nThere are 2 tables involved with this task. The name of the 1st table is actors, and the headers of this table are actorid,a_gender,a_quality. The name of the 2nd table is movies2actors, and the headers of this table are movieid,actorid,cast_num.',
 'There are 2 tables involved with this task. The name of the 1st table is actors, and the headers of this table are actorid,a_gender,a_quality. The name of the 2nd table is movies2actors, and the headers of this table are movieid,actorid,cast_num.')

In [50]:
# import random
# random.shuffle(task_and_environment)
task_and_environment[:10]
 

[('How many French drivers who obtain the laptime less than 01:00.00?\nThere are 2 tables involved with this task. The name of the 1st table is drivers, and the headers of this table are driverId,driverRef,number,code,forename,surname,dob,nationality,url. The name of the 2nd table is lapTimes, and the headers of this table are raceId,driverId,lap,position,time,milliseconds.',
  'There are 2 tables involved with this task. The name of the 1st table is drivers, and the headers of this table are driverId,driverRef,number,code,forename,surname,dob,nationality,url. The name of the 2nd table is lapTimes, and the headers of this table are raceId,driverId,lap,position,time,milliseconds.'),
 ('What was the longest time a driver had ever spent at a pit stop?\nThe name of this table is pitStops, and the headers of this table are raceId,driverId,stop,lap,time,duration,milliseconds.',
  'The name of this table is pitStops, and the headers of this table are raceId,driverId,stop,lap,time,duration,mil

In [38]:
from dbbench_more_tasks import more_task_and_environment2, more_task_and_environment1
# task_and_environment = more_task_and_environment2
task_and_environment = more_task_and_environment1

In [51]:
with open('dbbench-more_task_environment_list.pkl', 'rb') as file:
    task_and_environment = pickle.load(file)

In [52]:
# random.shuffle(task_and_environment)
len(task_and_environment)

475

In [53]:
task_and_environment[200]

("Insert a new employee named John Smith with a base salary of $75,000 into the payroll. His start date is January 1, 2023, with a position title of 'HR Manager'.",
 "The name of the table is 'Employees', and the columns are EmployeeID, FirstName, LastName, Position, BaseSalary, and StartDate.")

### Create Training Data Conversations

In [54]:
len(task_and_environment)

475

In [56]:
from tqdm.auto import tqdm
num_turns = 20

all_agents = []
all_environments = []
start = 0

for x in tqdm(range(start,start+len(task_and_environment))): 
    task = task_and_environment[x][0]
    environment_info = task_and_environment[x][1]
    print("Task {}:".format(x), task)
    agent_messages = [
        HumanMessage(content=mysql_agent_prompt),
        AIMessage(content="Ok."),
        HumanMessage(content=f"{task}. {environment_info}")
    ]
    agent_response = conv_model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_messages.append(agent_response)
    first_sql_block = re.search(r"```sql(.*?)```", agent_response.content, re.DOTALL)
    if first_sql_block:
        sql_code = first_sql_block.group(1).strip()
    else:
        sql_code = ""
    environment_prompt = environment_prompt_template.format(environment_info, sql_code)
    environment_messages = [
        HumanMessage(content=environment_prompt)
    ]
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    agent_messages.append(HumanMessage(content=environment_result.content))
    print(environment_result.content)
    SAVE = True
    for i in range(num_turns):
        try:
            agent_response = conv_model.predict_messages(agent_messages)
            print(agent_response.content)
            first_sql_block = re.search(r"```sql(.*?)```", agent_response.content, re.DOTALL)
            if first_sql_block:
                sql_code = first_sql_block.group(1).strip()
            else:
                sql_code = ""
            environment_messages.append(HumanMessage(content=sql_code))
            agent_messages.append(agent_response)
            if "Final Answer:" in agent_response.content:
                break
            environment_result = model.predict_messages(environment_messages)
            environment_messages.append(environment_result)
            print(environment_result.content)
            agent_messages.append(HumanMessage(content=environment_result.content))
        except:
            print("#################BadRequestError##################")
            SAVE = False
            break
    if SAVE:
        all_agents.append(deepcopy(agent_messages))
        all_environments.append(deepcopy(environment_messages))

  0%|          | 0/475 [00:00<?, ?it/s]

Task 0: Query to find the total sales made by the employee 'John Doe'.
In order to find the total sales made by the employee 'John Doe', we first need to find out the 'emp_id' of 'John Doe' from the 'employees' table. Then, we can use this 'emp_id' to find out all the sales made by 'John Doe' in the 'sales' table. We will sum up the 'total_price' of all these sales to get the total sales made by 'John Doe'. 

The first step is to find out the 'emp_id' of 'John Doe'. So, I will execute a SQL statement to select the 'emp_id' from the 'employees' table where 'first_name' is 'John' and 'last_name' is 'Doe'. Here is the SQL statement:

Action: Operation
```sql
SELECT emp_id FROM employees WHERE first_name='John' AND last_name='Doe';
```
[('123',)]
From the output, we know that the 'emp_id' of 'John Doe' is '123'. 

Next, we need to find out all the sales made by 'John Doe' in the 'sales' table. We will select the 'total_price' from the 'sales' table where 'emp_id' is '123' and then sum them

APIConnectionError: Connection error.

In [57]:
agent_messages

[HumanMessage(content='I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question. You have to explain the problem and your solution to me and write down your thoughts. After thinking and explaining thoroughly, every round you can choose to operate or to answer. your operation should be like this: Action: Operation ```sql SELECT * FROM table WHERE condition; ``` You MUST put SQL in markdown format without any other comments. Your SQL should be in one line. Every time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you and give you the output. If you are done operating, and you want to commit your final answer, then write down: Action: Answer Final Answer: ["ANSWER1", "ANSWER2", ...] DO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer. Your answer should be accurate. Your answer must

In [60]:
all_agents[-1]

[HumanMessage(content='I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question. You have to explain the problem and your solution to me and write down your thoughts. After thinking and explaining thoroughly, every round you can choose to operate or to answer. your operation should be like this: Action: Operation ```sql SELECT * FROM table WHERE condition; ``` You MUST put SQL in markdown format without any other comments. Your SQL should be in one line. Every time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you and give you the output. If you are done operating, and you want to commit your final answer, then write down: Action: Answer Final Answer: ["ANSWER1", "ANSWER2", ...] DO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer. Your answer should be accurate. Your answer must

In [61]:
all_environments[-1]

[HumanMessage(content="\nPretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\nThere are 2 tables involved with this task. The name of the first table is HospitalInfo, with headers: HospitalID, HospitalName, HospitalAddress, HospitalCity, HospitalState, HospitalZipcode. The second table is PatientInfo, with headers: PatientID, HospitalID (refers to HospitalInfo), PatientName, PatientDOB, PatientAdmissionDate, PatientDischargeDate, PatientCondition, PatientRoomType (ICU, General, etc.).\n\nFirst command:\nSELECT * FROM HospitalInfo LIMIT 10;\n\nP

In [63]:
print(len(all_agents), len(all_environments))

446 446


In [64]:
conversations_ = []
for chat in all_agents:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )
        

In [65]:
with open('./synthetic-0/dbbench-interactions-syn_env_syn_chat-raw-5.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")